# 01 Data Collection – Forex Price Prediction Project

## Objectives

* Fetch historical OHLCV (Open, High, Low, Close, Volume) data for 10 selected forex pairs using the OANDA API.
* Save the raw data in CSV format in the `data/raw/` directory for further processing.

## Inputs

* OANDA API Key stored securely in `.env` file.
* Selected forex pairs:
  1. GBP/USD  
  2. EUR/GBP  
  3. EUR/USD  
  4. AUD/USD  
  5. NZD/USD  
  6. USD/JPY  
  7. USD/SGD  
  8. USD/CHF  
  9. USD/CAD  
  10. GBP/CAD
* Desired number of historical candles per pair and timeframe (e.g., last 1000 H4 candles).

## Outputs

* 10 CSV files stored in `data/raw/` named as:
  * GBP_USD_forex_data.csv
  * EUR_GBP_forex_data.csv
  * EUR_USD__forex_data.csv
  * AUD_USD_forex_data.csv
  * NZD_USD_forex_data.csv
  * USD_JPY_forex_data.csv
  * USD_SGD_forex_data.csv
  * USD_CHF_forex_data.csv
  * USD_CAD_forex_data.csv
  * GBP_CAD_forex_data.csv 

* Each CSV contains:
  - `timestamp`  
  - `open`  
  - `high`  
  - `low`  
  - `close`  
  - `volume`  

## Additional Comments

* This notebook covers the **Data Collection** step of CRISP-DM.
* The raw files will be used in **02_data_cleaning_feature_engineering.ipynb** for feature creation.
* Ensure that the `.env` file with `OANDA_API_KEY` is correctly configured before running this notebook.


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
import os

# Change to project root if running notebook from 'notebooks/' folder
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")

print("Current working directory:", os.getcwd())


Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Import Libraries & Load API Key

Purpose: Import dependencies and load OANDA API key securely.

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve OANDA API key
OANDA_API_KEY = os.getenv("OANDA_API_KEY")

# Verify that the key is loaded
print("OANDA_API_KEY loaded:", bool(OANDA_API_KEY))


## Define Forex Pairs for Data Collection

We will collect **H4 (4-hour)** candle data for the following 10 forex pairs.  
The raw data will be saved in the `data/raw/` folder for future preprocessing.


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
